In [ ]:
import torch
import matplotlib.pyplot as plt
import warnings
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from functions import get_dataset, get_model_accuracy, plot_chart
from constants import DEVICE
from multiagent import WeightedAverage, Plurality, ProbabilitiesSum, Borda, MaxProb
from tqdm import tqdm
from sklearn.metrics import ConfusionMatrixDisplay


# Libraries setup (and ignoring warnings)
warnings.simplefilter(action='ignore', category=FutureWarning)
tqdm.pandas()

# Loading the models (from local machine) and tokenizers (from HuggingFace hub)
tokenizer_xlnet = AutoTokenizer.from_pretrained('models/xlnet')
model_xlnet = AutoModelForSequenceClassification.from_pretrained('models/xlnet', num_labels = 3).to(DEVICE)
tokenizer_bert = AutoTokenizer.from_pretrained('models/bert')
model_bert = AutoModelForSequenceClassification.from_pretrained('models/bert', num_labels = 3).to(DEVICE)
tokenizer_gpt2 = AutoTokenizer.from_pretrained('models/gpt2')
model_gpt2 = AutoModelForSequenceClassification.from_pretrained('models/gpt2', num_labels = 3).to(DEVICE)

# Getting the test data
df = get_dataset('dataset_test.csv')

In [ ]:
# Calculating each model accuracy on the merged test set
gpt2_all_acc, gpt2_cm = get_model_accuracy(model_gpt2, tokenizer_gpt2, df)
bert_all_acc, bert_cm = get_model_accuracy(model_bert, tokenizer_bert, df)
xlnet_all_acc, xlnet_cm = get_model_accuracy(model_xlnet, tokenizer_xlnet, df)

print("GPT2", gpt2_all_acc)
print("BERT", bert_all_acc)
print("XLNET", xlnet_all_acc)

In [ ]:
disp_gpt2_cm = ConfusionMatrixDisplay(confusion_matrix=gpt2_cm, display_labels=['0','1','2'])
disp_bert_cm = ConfusionMatrixDisplay(confusion_matrix=bert_cm, display_labels=['0','1','2'])
disp_xlnet_cm = ConfusionMatrixDisplay(confusion_matrix=xlnet_cm, display_labels=['0','1','2'])

# Plotting the result
plot_chart(['GPT2','BERT','XLNet'], [gpt2_all_acc, bert_all_acc, xlnet_all_acc], 'Model accuracies on test set')

fig, axes = plt.subplots(3, 1, figsize=(10, 15))

axes[0].set_title('GPT-2 Confusion Matrix - All dataset')
axes[1].set_title('BERT Confusion Matrix - All dataset')
axes[2].set_title('XLNET Confusion Matrix - All dataset')

disp_gpt2_cm.plot(cmap=plt.cm.Blues, ax=axes[0])
disp_bert_cm.plot(cmap=plt.cm.Blues, ax=axes[1])
disp_xlnet_cm.plot(cmap=plt.cm.Blues, ax=axes[2])

plt.tight_layout()
plt.show()

In [ ]:
# Initializing all the multiagent voting rules
rules = [ProbabilitiesSum(), Plurality(), MaxProb(), WeightedAverage(), Borda()]

# Calculating all the multiagent results
probabilities = {}
for index, row in tqdm(df.iterrows(), total=len(df)):
    # Calculating GPT2's prediction
    inputs_gpt2    = tokenizer_gpt2(row['text'], return_tensors="pt", max_length=128, truncation=True).to(DEVICE)
    probabilities['gpt2']   = torch.softmax(model_gpt2(**inputs_gpt2).logits, 1)

    # Calculating XLNET's prediction
    inputs_xlnet    = tokenizer_xlnet(row['text'], return_tensors="pt", max_length=128, truncation=True).to(DEVICE)
    probabilities['xlnet']   = torch.softmax(model_xlnet(**inputs_xlnet).logits, 1)

    # Calculating BERT's prediction
    inputs_bert    = tokenizer_bert(row['text'], return_tensors="pt", max_length=128, truncation=True).to(DEVICE)
    probabilities['bert']   = torch.softmax(model_bert(**inputs_bert).logits, 1)

    # Applying each voting rule
    for r in rules:
        r(probabilities, row['labels'])

# Plotting the results
plot_chart([r.__class__.__name__ for r in rules], [ r.correct_counter/len(df) for r in rules], 'Multiagent accuracies')

for r in rules:
    print(r.__class__.__name__, r.correct_counter/len(df))
